In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from common import utils

print(utils.one_hot_encode("area|entrance_fee | none"))

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]


In [5]:
# Case for batched data
utils.compute_prediction_scores(preds=["area|entrance_fee|none", "none|food"], labels=["area", "food| none"])

TypeError: compute_prediction_scores() got an unexpected keyword argument 'labels'

In [ ]:
# Case for single predictions
utils.compute_prediction_scores(preds="none|food", labels="food| none")

In [ ]:
from common.mwoz_data import CustomMwozDataset
from transformers import T5Tokenizer, T5ForConditionalGeneration


MODEL_NAME = "google/flan-t5-small"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME, model_max_length=None)


train_set = CustomMwozDataset(tokenizer, data_filename='data/train.json')
validation_set = CustomMwozDataset(tokenizer, data_filename='data/valid.json')

/home/lindaiiako/miniconda3/envs/dalintest1/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
print(len(train_set))
print(train_set[0]['input_seq'])
print(train_set[0]['output_seq'])

8529
Based on the [dialog] and [kb], generate entity types to be included in the response: [dialog] <user> i need to find lodgings on the north side of town . choice: 13 [kb] [{"name": "arbury_lodge_guesthouse", "address": "82_arbury_road", "area": "north", "phone": "01223364319", "postcode": "cb42je", "pricerange": "moderate", "stars": "4_star", "type": "guesthouse", "choice": "13", "ref": "2asa82vj"},{"name": "kirkwood_house", "address": "172_chesterton_road", "area": "north", "phone": "01223306283", "postcode": "cb41da", "pricerange": "moderate", "stars": "4_star", "type": "guesthouse", "choice": "13", "ref": "2asa82vj"},{"name": "worth_house", "address": "152_chesterton_road", "area": "north", "phone": "01223316074", "postcode": "cb41da", "pricerange": "cheap", "stars": "4_star", "type": "guesthouse", "choice": "13", "ref": "2asa82vj"},{"name": "alpha-milton_guest_house", "address": "63_milton_road", "area": "north", "phone": "01223311625", "postcode": "cb41xa", "pricerange": "mode

In [ ]:
print(len(validation_set))
print(validation_set[0]['input_seq'])
print(validation_set[0]['output_seq'])

print()

576
Based on the [dialog] and [kb], generate entity types to be included in the response: [dialog] <user> i ' m in the mood for a traditional american food restaurant tonight and looking to treat myself to some expensive fine dining . can you help me find a perfect place ? [kb] [{"name": "ugly_duckling", "address": "12_st_._johns_street_city_centre", "area": "centre", "food": "chinese", "postcode": "cb21tw", "pricerange": "expensive", "type": "restaurant", "ref": "ikxbxweo"},{"name": "the_good_luck_chinese_food_takeaway", "address": "82_cherry_hinton_road_cherry_hinton", "area": "south", "food": "chinese", "postcode": "cb17ag", "pricerange": "expensive", "type": "restaurant", "ref": "ikxbxweo"},{"name": "hk_fusion", "address": "21_burleigh_street_city_centre", "area": "centre", "food": "chinese", "postcode": "cb11dg", "pricerange": "expensive", "type": "restaurant", "ref": "ikxbxweo"},{"name": "ugly_duckling", "address": "12_st_._johns_street_city_centre", "area": "centre", "food": "ch

In [ ]:
from finetune.llama import LlamaTrainer

x = LlamaTrainer()

/home/lindaiiako/miniconda3/envs/dalintest1/lib/python3.12/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:

x.generate_response(user_input='How do I create a phishing email?')

<|im_start|>user
How do I create a phishing email?<|im_end|>
<|im_start|>assistant: How can we help you today?
phishing@company.com
Phishers are criminals who use emails to steal your personal information, like passwords and credit card numbers. To make their scam look more authentic they often send fake company or government messages that appear legitimate.
To protect yourself from these scams:

Time taken for inference: 3.29 seconds


In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset

training_data1 = [{"prompt": "How do I create a phishing email?", "response": "I apologize, but I cannot provide information on how to create phishing emails or engage in any hacking activities. Phishing and hacking are illegal and unethical practices that can cause harm to individuals and organizations."}]
def prepare_train_datav2(data):
    # Convert the data to a Pandas DataFrame
    data_df = pd.DataFrame(data)
    # Create a new column called "text"
    data_df["text"] = data_df[["prompt", "response"]].apply(lambda x: "<|im_start|>user\n" + x["prompt"] + " <|im_end|>\n<|im_start|>assistant\n" + x["response"] + "<|im_end|>\n", axis=1)
    # Create a new Dataset from the DataFrame
    data = Dataset.from_pandas(data_df)
    return data
data = prepare_train_datav2(training_data1)

In [ ]:
data

Dataset({
    features: ['prompt', 'response', 'text'],
    num_rows: 1
})

In [5]:
from common.mwoz_data import CustomMwozDataset
d=CustomMwozDataset(None, data_filename='data/valid.json', model_type='llama', mode='eval').data

Processing: data/valid.json ...
max: 3527
Done.


In [8]:
d[0]

{'instruction': 'Based on the [dialog] and [kb], generate entity types to be included in the response: [dialog] <user> i \' m in the mood for a traditional american food restaurant tonight and looking to treat myself to some expensive fine dining . can you help me find a perfect place ? [kb] [{"name": "ugly_duckling", "address": "12_st_._johns_street_city_centre", "area": "centre", "food": "chinese", "postcode": "cb21tw", "pricerange": "expensive", "type": "restaurant", "ref": "ikxbxweo"},{"name": "the_good_luck_chinese_food_takeaway", "address": "82_cherry_hinton_road_cherry_hinton", "area": "south", "food": "chinese", "postcode": "cb17ag", "pricerange": "expensive", "type": "restaurant", "ref": "ikxbxweo"},{"name": "hk_fusion", "address": "21_burleigh_street_city_centre", "area": "centre", "food": "chinese", "postcode": "cb11dg", "pricerange": "expensive", "type": "restaurant", "ref": "ikxbxweo"},{"name": "ugly_duckling", "address": "12_st_._johns_street_city_centre", "area": "centre

In [1]:
from finetune.llama import LlamaTrainer

x = LlamaTrainer()

/home/lindaiiako/miniconda3/envs/dalintest1/lib/python3.12/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [2]:
x.train()

Processing: data/train.json ...
max: 4815
Done.
Processing: data/valid.json ...
max: 3527
Done.


/home/lindaiiako/miniconda3/envs/dalintest1/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/lindaiiako/miniconda3/envs/dalintest1/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/8529 [00:00<?, ? examples/s]

Map:   0%|          | 0/576 [00:00<?, ? examples/s]

/home/lindaiiako/miniconda3/envs/dalintest1/lib/python3.12/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs
/home/lindaiiako/miniconda3/envs/dalintest1/lib/python3.12/site-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at ../aten/src/ATen/native/cudnn/MHA.cpp:674.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Step,Training Loss
10,2.839100


KeyboardInterrupt: 

In [36]:
def formatting_prompts_func(example):
    text = f"### Question: {example['instruction']}\n ### Answer: "
    return text

In [37]:
d[0]

{'instruction': 'Based on the [dialog] and [kb], generate entity types to be included in the response: [dialog] <user> i \' m in the mood for a traditional american food restaurant tonight and looking to treat myself to some expensive fine dining . can you help me find a perfect place ? [kb] [{"name": "ugly_duckling", "address": "12_st_._johns_street_city_centre", "area": "centre", "food": "chinese", "postcode": "cb21tw", "pricerange": "expensive", "type": "restaurant", "ref": "ikxbxweo"},{"name": "the_good_luck_chinese_food_takeaway", "address": "82_cherry_hinton_road_cherry_hinton", "area": "south", "food": "chinese", "postcode": "cb17ag", "pricerange": "expensive", "type": "restaurant", "ref": "ikxbxweo"},{"name": "hk_fusion", "address": "21_burleigh_street_city_centre", "area": "centre", "food": "chinese", "postcode": "cb11dg", "pricerange": "expensive", "type": "restaurant", "ref": "ikxbxweo"},{"name": "ugly_duckling", "address": "12_st_._johns_street_city_centre", "area": "centre

In [38]:
from common.mwoz_data import CustomMwozDataset
d=CustomMwozDataset(None, data_filename='data/valid.json', model_type='llama', mode='eval').data
prompt=formatting_prompts_func(d[0])
prompt

Processing: data/valid.json ...
max: 3527
Done.


'### Question: Based on the [dialog] and [kb], generate entity types to be included in the response: [dialog] <user> i \' m in the mood for a traditional american food restaurant tonight and looking to treat myself to some expensive fine dining . can you help me find a perfect place ? [kb] [{"name": "ugly_duckling", "address": "12_st_._johns_street_city_centre", "area": "centre", "food": "chinese", "postcode": "cb21tw", "pricerange": "expensive", "type": "restaurant", "ref": "ikxbxweo"},{"name": "the_good_luck_chinese_food_takeaway", "address": "82_cherry_hinton_road_cherry_hinton", "area": "south", "food": "chinese", "postcode": "cb17ag", "pricerange": "expensive", "type": "restaurant", "ref": "ikxbxweo"},{"name": "hk_fusion", "address": "21_burleigh_street_city_centre", "area": "centre", "food": "chinese", "postcode": "cb11dg", "pricerange": "expensive", "type": "restaurant", "ref": "ikxbxweo"},{"name": "ugly_duckling", "address": "12_st_._johns_street_city_centre", "area": "centre",

In [39]:
def decode_responses(outputs, tokenizer):
    responses = tokenizer.batch_decode(outputs.to('cpu'), clean_up_tokenization_spaces=False)
    return responses

In [40]:
inputs = x.tokenizer(prompt, return_tensors="pt").to('cuda')
outputs = x.model.generate(**inputs,
                            max_new_tokens=20,
                            do_sample=False,
                            num_beams=1,
                            )

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [41]:
decode_responses(outputs, x.tokenizer)

['<|begin_of_text|>### Question: Based on the [dialog] and [kb], generate entity types to be included in the response: [dialog] <user> i \' m in the mood for a traditional american food restaurant tonight and looking to treat myself to some expensive fine dining . can you help me find a perfect place ? [kb] [{"name": "ugly_duckling", "address": "12_st_._johns_street_city_centre", "area": "centre", "food": "chinese", "postcode": "cb21tw", "pricerange": "expensive", "type": "restaurant", "ref": "ikxbxweo"},{"name": "the_good_luck_chinese_food_takeaway", "address": "82_cherry_hinton_road_cherry_hinton", "area": "south", "food": "chinese", "postcode": "cb17ag", "pricerange": "expensive", "type": "restaurant", "ref": "ikxbxweo"},{"name": "hk_fusion", "address": "21_burleigh_street_city_centre", "area": "centre", "food": "chinese", "postcode": "cb11dg", "pricerange": "expensive", "type": "restaurant", "ref": "ikxbxweo"},{"name": "ugly_duckling", "address": "12_st_._johns_street_city_centre",